# Introduction

In [1]:
from IPython.display import display
import ipywidgets as widgets
from modelscope import snapshot_download
from sherpa_onnx import OnlineRecognizer

from ipyaudio import Recorder

repo_dir = snapshot_download("pengzhendong/streaming-paraformer-zh-en")
recognizer = OnlineRecognizer.from_paraformer(
    tokens=f"{repo_dir}/tokens.txt",
    encoder=f"{repo_dir}/encoder.onnx",
    decoder=f"{repo_dir}/decoder.onnx",
    num_threads=1,
    provider="cpu",
    sample_rate=16000,
    decoding_method="greedy_search",
)
stream = recognizer.create_stream()

2025-04-04 20:53:35,657 - modelscope - INFO - Not logged-in, you can login for uploadingor accessing controlled entities.


In [2]:
recorder = Recorder()

def on_end_change(change):
    recognizer.reset(stream)
    # stream.input_finished()
recorder.observe(on_end_change, names="end")

Recorder(value=None, config={'options': {'backend': 'WebAudio', 'container': '', 'waveColor': '#4BF2A7'}, 'rec…

In [10]:
display_id = display("", display_id=True)
stream = recognizer.create_stream()

def on_frame_change(change):
    stream.accept_waveform(16000, recorder.frame[0].tolist())
    while recognizer.is_ready(stream):
        recognizer.decode_stream(stream)
        result = recognizer.get_result(stream)
        display_id.update(result)

recorder.observe(on_frame_change, names="frame")

'一二三四五六七八九十十十一十二十三十四二二三四'